Some examples using the excellent Plotly Library

# 3D Surface Plot

In [1]:
# (*) To communicate with Plotly's server, sign in with credentials file
import plotly.plotly as py

# (*) Useful Python/Plotly tools
import plotly.tools as tls

# (*) Graph objects to piece together plots
from plotly.graph_objs import *

import numpy as np  # (*) numpy for math functions and arrays

In [2]:
# (*) Import the math functions needed in this cell
from numpy import pi, cos, exp

# Define the function to be plotted
def fxy(x, y):
    A = 1  # choose a maximum amplitude 
    return A*(cos(pi*x*y))**2 * exp(-(x**2+y**2)/2.)

In [3]:
# Choose length of square domain, make row and column vectors
L = 4
x = y = np.arange(-L/2., L/2., 0.1)  # use a mesh spacing of 0.1
yt = y[:, np.newaxis]  # (!) make column vector

# Get surface coordinates!
z = fxy(x, yt)

In [4]:
trace1 = Surface(
    z=z,  # link the fxy 2d numpy array
    x=x,  # link 1d numpy array of x coords
    y=y   # link 1d numpy array of y coords
)

# Package the trace dictionary into a data object
data = Data([trace1])

In [5]:
# Dictionary of style options for all axes
axis = dict(
    showbackground=True, # (!) show axis background
    backgroundcolor="rgb(204, 204, 204)", # set background color to grey
    gridcolor="rgb(255, 255, 255)",       # set grid line color
    zerolinecolor="rgb(255, 255, 255)",   # set zero grid line color
)

# Make a layout object
layout = Layout(
    title='$f(x,y) = A \cos(\pi x y) e^{-(x^2+y^2)/2}$', # set plot title
    scene=Scene(  # (!) axes are part of a 'scene' in 3d plots
        xaxis=XAxis(axis), # set x-axis style
        yaxis=YAxis(axis), # set y-axis style
        zaxis=ZAxis(axis)  # set z-axis style
    )
)

In [6]:
# Make a figure object
fig = Figure(data=data, layout=layout)

# (@) Send to Plotly and show in notebook
py.iplot(fig, filename='s8_surface')

# Fractal Trees

In [14]:
from math import pi as PI
from math import sin, cos
import random

from plotly.widgets import GraphWidget
from plotly.graph_objs import *
import plotly.plotly as py
import plotly.tools as tls

from IPython.html import widgets 
from IPython.display import display, clear_output

root = 12


In [15]:
# color the tree with a gradient from root_col to tip_col
# interpolate linearly to get color at a given position in the gradient
def get_col(root_col, tip_col, iterat):
    r = ((iterat*1.0/root)*(root_col[0]-tip_col[0]))+tip_col[0]
    g = ((iterat*1.0/root)*(root_col[1]-tip_col[1]))+tip_col[1]
    b = ((iterat*1.0/root)*(root_col[2]-tip_col[2]))+tip_col[2]
    return '#%02x%02x%02x' % (r,g,b)
    
def tree_graph(iterat=12, branch_angle=18.0):
    # angle to radian factor
    ang2rad = PI/180.0
    # experiment with trunk length (try 120)
    t = 120
    # experiment with factor to contract the trunk each iteratation (try 0.7)
    r = 0.7
    # starting orientation (initial 90 deg)
    theta = 90.0 * ang2rad
    # experiment with angle of the branch (try 18 deg)
    dtheta = branch_angle * ang2rad
    # experiment with gradient color choices
    root_col = (40,40,40)
    tip_col = (250,250,250)
    # experiment with factor to increase random angle variation as child branches get smaller
    iterscale = 6.0
    # center of bottom
    origin = (250, 500)
    root=iterat
    # make the tree
    
    def fractal_tree(lines, iterat, origin, t, r, theta, dtheta, root_col, tip_col, randomize=False):
        """
        draws branches
        iterat:     iteratation number, stop when iterat == 0
        origin:   x,y coordinates of the start of this branch
        t:        current trunk length
        r:        factor to contract the trunk each iteratation
        theta:    starting orientation
        dtheta:   angle of the branch
        """
        if iterat == 0:
            return lines
        # render the branch
        x0, y0 = origin

        # randomize the length
        randt = random.random()*t
        if randomize:
            x, y = x0 + randt * cos(theta), y0 - randt * sin(theta)
        else:
            x, y = x0 + cos(theta), y0 - sin(theta)
        # color the branch according to its position in the tree
        col = get_col(root_col, tip_col, iterat)
        # add to traces
        lines.append(Scatter(x=[x0, x], y=[y0, y], mode='lines', line=Line(color=col, width=1)))
        # recursive calls
        if randomize:
            fractal_tree(lines, iterat-1, (x,y), t * r, r, theta + (random.random())*(iterscale/(iterat+1))*dtheta, dtheta, root_col, tip_col, randomize)
            fractal_tree(lines, iterat-1, (x,y), t * r, r, theta - (random.random())*(iterscale/(iterat+1))*dtheta, dtheta, root_col, tip_col, randomize) 
        else: 
            fractal_tree(lines, iterat-1, (x,y), t * r, r, theta + dtheta, dtheta, root_col, tip_col, randomize)
            fractal_tree(lines, iterat-1, (x,y), t * r, r, theta - dtheta, dtheta, root_col, tip_col, randomize)
    
    lines = []
    fractal_tree(lines, iterat, origin, t, r, theta, dtheta, root_col, tip_col, True)
    
    # group the lines by similar color
    branches = {}
    for line in lines:
        color = line['line']['color']
        if color not in branches:
            branches[color] = line
        else:
            branches[color]['x'].extend(line['x'])
            branches[color]['y'].extend(line['y'])
            branches[color]['x'].append(None)
            branches[color]['y'].append(None)
            
    
    branch_data = [branches[c] for c in branches]
    
    return branch_data

In [16]:
g = GraphWidget('https://plot.ly/~chris/4103')

iterations = widgets.IntSliderWidget()
iterations.min= 2
iterations.max= 14
iterations.value = 12
iterations.description = 'iterations'

branch_angle = widgets.IntSliderWidget()
branch_angle.min = 5
branch_angle.max = 90
branch_angle.value = 18
branch_angle.description = 'branch angle'

seed = widgets.ButtonWidget()
seed.description = 'Grow a new tree'

def regraph_tree(_):
    branch_data = tree_graph(iterations.value, branch_angle.value)

    g.restyle({'x': [[]], 'y': [[]]})
    g.add_traces(branch_data)

iterations.on_trait_change(regraph_tree, 'value')
branch_angle.on_trait_change(regraph_tree, 'value')
seed.on_click(regraph_tree)
    
display(iterations)
display(branch_angle)
display(seed)
display(g)

In [10]:
layout = Layout(yaxis=YAxis(autorange='reversed'), width=500, showlegend=False)

g.relayout(layout)

# Widget 3D

In [11]:
from plotly.widgets import GraphWidget

from IPython.html import widgets 
from IPython.display import display, clear_output
import numpy as np

In [12]:
g = GraphWidget('https://plot.ly/~jackp/2590')
x = y = np.arange(-5,5,0.1)
yt = x[:,np.newaxis]

class z_data:
    def __init__(self):
        self.z = np.cos(x*yt)+np.sin(x*yt)
    
    def on_z_change(self, name, old_value, new_value):
        self.z = np.cos(x*yt*(new_value+1)/100)+np.sin(x*yt*(new_value+1/100))
        self.replot()
        
    def replot(self):
        g.restyle({ 'z': [self.z] })

z_slider = widgets.FloatSliderWidget(min=0,max=3,value=1,step=0.05)
z_slider.description = 'Frequency'
z_slider.value = 1

z_state = z_data()
z_slider.on_trait_change(z_state.on_z_change, 'value')

In [13]:
display(z_slider)
display(g)